In [ ]:
import pandas as pd
import numpy as np
import spacy
import re
from transformers import pipeline

In [ ]:
import pandas as pd

df = pd.read_csv("final_csv.csv")

In [ ]:
df.head(3)

,title,link,description
0,"A new development in Gainesville, Va., provide...",https://www.washingtonpost.com/realestate/a-ta...,I am Chief Investment Officer of Charles Stanl...
1,"A new development in Gainesville, Va., provide...",https://www.washingtonpost.com/realestate/a-ta...,"SPRINGFIELD, Ill. (AP) - Officials say prelimi..."
2,"A new development in Gainesville, Va., provide...",https://www.washingtonpost.com/realestate/a-ta...,The eighth round of North American Free Trade ...


In [ ]:
print(df.columns)

Index(['title', 'link', 'description'], dtype='object')


In [ ]:
def extract_geolocation(text):
    nlp = spacy.load("en_core_web_sm")

    doc = nlp(text)

    location_labels = {"GPE", "LOC"}


    extracted_locations = []
    locations = set()

    for ent in doc.ents:
        if ent.label_ in location_labels:
           locations.add(ent.text)


    extracted_locations.extend(list(locations))

    if not extracted_locations:
      return["NA"]


    return extracted_locations

In [ ]:
text = '''I live in New York,and my name is Mumbai but I work in Paris.'''
result = extract_geolocation(text)
print(result)

['New York', 'Mumbai', 'Paris']


In [ ]:
def add_geolocation(row):
    description = row['description']

    description_geolocation = extract_geolocation(description)

    row["desc_geolocation"] = description_geolocation

    return row

In [ ]:
df_with_geolocation = df.apply(add_geolocation, axis=1)

In [ ]:
df_with_geolocation['desc_geolocation'] = df_with_geolocation['desc_geolocation'].apply(np.array)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, classification_report
import pickle

# Assuming df_with_geolocation is your DataFrame with 'description' and 'desc_geolocation' columns
# You need to have a labeled dataset where 'desc_geolocation' contains a list of labels

# Convert the multi-label target variable into a binary matrix
mlb = MultiLabelBinarizer()
labels_matrix = mlb.fit_transform(train_df['desc_geolocation'])

train_df, test_df = train_test_split(df_with_geolocation, test_size=0.2, random_state=42)

model = make_pipeline(TfidfVectorizer(), MultiOutputClassifier(MultinomialNB()))

# Train the model
model.fit(train_df['description'], labels_matrix)

test_labels_matrix = mlb.transform(test_df['desc_geolocation'])
predictions = model.predict(test_df['description'])

predicted_labels = mlb.inverse_transform(predictions)

# Evaluate the model
accuracy = accuracy_score(test_labels_matrix, predictions)
print(f"Model Accuracy: {accuracy}")

with open('geolocation_model.pkl', 'wb') as file:
    pickle.dump(model, file)

with open('multilabel_binarizer.pkl', 'wb') as file:
    pickle.dump(mlb, file)

# print("Classification Report:")
# print(classification_report(test_labels_matrix, predictions))


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['Australia', 'Ayton', 'Belgium', 'Beverly Hills', 'Birmingham City', 'Bollywood', 'Clacton', 'Coincheck', 'Columbus', 'Connecticut', 'Delft', 'Digital Drama Series', 'Dropbox', 'Dundalk', 'Espoo', 'Finland', 'Formaldehyde', 'Gulf', 'Igbo', 'Israel', 'Kemira', 'Lagos', 'Las Vegas', 'Lebanon', 'Limpopo', 'Louisville', 'N.M.', 'New Mexico', "New Mexico's", 'Nigeria', 'Nollywood', 'Northern Europe', 'Oregon', 'Qatar', 'River North', 'Rockingham', 'SAN DIEGO', 'Salalah', "Saudi Arabia's", 'Singapore', 'South Ingleside Avenue', 'Southampton', 'Sr.', 'States', 'Uruguay', 'WA', 'Wis.', 'Woodman Point', 'iOS', 'the North West', 'the Safety Bay', 'the United States of America'] will be ignored
  warnings.warn(


Model Accuracy: 0.275
